In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [1]:
import torch
print(torch.__version__)


2.5.1+cu118


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [3]:
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

NVIDIA GeForce RTX 4060 Laptop GPU


In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("masoudnickparvar/brain-tumor-mri-dataset")

print("Path to dataset files:", path)

d:\Programming\Cancer_Detection_Model\.env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\sande\.cache\kagglehub\datasets\masoudnickparvar\brain-tumor-mri-dataset\versions\1


In [6]:
import os
print("Files in the dataset directory:", os.listdir(path))

Files in the dataset directory: ['Testing', 'Training']


In [47]:
# ['glioma', 'meningioma', 'notumor', 'pituitary']

In [48]:
try:
    # Path to the 'Training' folder
    training_folder_path = os.path.join(path, 'Training')

    # Path to the 'glioma_tumor' subfolder
    glioma_folder_path = os.path.join(training_folder_path, 'glioma_tumor')

    # Get the name of the first image file in that folder
    sample_image_name = os.listdir(glioma_folder_path)[0]

    # Create the full path to that single image
    full_image_path = os.path.join(glioma_folder_path, sample_image_name)

    print(f"\nAttempting to read image: {full_image_path}")


    # --- Step 3: Open and display the image ---
    # Use Pillow's Image.open() to read the file
    img = Image.open(full_image_path)
    

    # You can now work with the image (e.g., get its properties)
    print(f"\nImage loaded successfully!")
    print(f"  Format: {img.format}")
    print(f"  Size: {img.size}")
    print(f"  Mode: {img.mode}")

    # Use matplotlib to display the image in a window or notebook
    plt.imshow(img)
    plt.title(f"Sample: {sample_image_name}")
    plt.axis('off') # Hides the x and y axis labels
    plt.show()

except FileNotFoundError:
    print("\nError: Could not find the expected folder structure.")
    print("Please check the downloaded files and adjust the paths.")
except Exception as e:
    print(f"\nAn error occurred: {e}")


Error: Could not find the expected folder structure.
Please check the downloaded files and adjust the paths.
